In [4]:
# 이 셀을 가장 먼저 실행해야 'pd' 에러가 사라집니다!
import torch
import torch.nn as nn
import pandas as pd          # <--- 얘가 있어야 'pd'를 알 수 있어요
import selfies as sf
from tqdm.auto import tqdm
import math
import os
import numpy as np

# 작업 경로 고정 (아까 드린 코드)
try:
    PROJECT_DIR = os.path.expanduser("~/data/interns/lsw0927/transgem")
    os.chdir(PROJECT_DIR)
    print(f"✅ 작업 경로: {os.getcwd()}")
except:
    pass

✅ 작업 경로: /home/lsw0927/data/interns/lsw0927/transgem


/home/lsw0927/miniconda3/envs/transgem/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# --- 2. 설정 및 클래스 정의 (이 셀을 꼭 실행해야 합니다!) ---

import math

# 설정 정의
CONFIG = {
    "data_path": "./TransGEM/data/subLINCS.csv",
    # 체크포인트 파일명이 맞는지 다시 확인!
    "model_path": "checkpoints/model_epoch_200.pt", 
    "output_file": "generated_molecules_PC3.txt",
    "d_model": 64,
    "nhead": 8,
    "num_layers": 6,
    "ff_dim": 512,
    "max_len": 100,
    "gene_dim": 978 * 10
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device settings: {DEVICE}")

# --- 모델 및 헬퍼 함수 정의 ---

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=500):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TransGEM(nn.Module):
    def __init__(self, num_genes_dim, num_cells, vocab_size, d_model=64, nhead=8, num_layers=6, dim_feedforward=512):
        super(TransGEM, self).__init__()
        self.d_model = d_model
        self.cell_embedding = nn.Embedding(num_cells, d_model)
        self.gene_linear = nn.Sequential(
            nn.Linear(num_genes_dim, d_model), nn.ReLU(), nn.Linear(d_model, d_model)
        )
        self.fusion_layer = nn.Sequential(
            nn.Linear(d_model * 2, d_model), nn.ReLU(), nn.Linear(d_model, d_model)
        )
        self.mol_embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.generator = nn.Linear(d_model, vocab_size)

    def forward(self, cell_idx, gene_tensor, tgt):
        cell_embed = self.cell_embedding(cell_idx)
        gene_embed = self.gene_linear(gene_tensor)
        context = torch.cat([cell_embed, gene_embed], dim=-1)
        memory = self.fusion_layer(context).unsqueeze(1)
        tgt_embed = self.mol_embedding(tgt) * math.sqrt(self.d_model)
        tgt_embed = self.pos_encoder(tgt_embed)
        sz = tgt.size(1)
        mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1).to(tgt.device)
        output = self.transformer_decoder(tgt=tgt_embed, memory=memory, tgt_mask=mask)
        return self.generator(output)

def build_vocab(smiles_list):
    vocab = set()
    print("Building Vocabulary...")
    for smiles in tqdm(smiles_list, desc="Tokenizing"):
        try:
            selfie = sf.encoder(smiles)
            if selfie is None: continue
            tokens = list(sf.split_selfies(selfie))
            vocab.update(tokens)
        except: continue
    vocab = sorted(list(vocab))
    token2idx = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
    for i, token in enumerate(vocab): token2idx[token] = i + 3
    return token2idx

def parse_gene_string(gene_str):
    if isinstance(gene_str, str):
        try: return [float(x) for x in gene_str.split('//') if x.strip()]
        except: return []
    return []

def tenfold_binary_embedding(value):
    try: value = float(value)
    except: value = 0.0
    sign_bit = 1 if value > 0 else 0
    tenfold_value = int(abs(value) * 10)
    binary_str = bin(tenfold_value)[2:].zfill(9)
    if len(binary_str) > 9: binary_str = binary_str[-9:] 
    return [sign_bit] + [int(b) for b in binary_str]

Device settings: cuda


In [7]:
# --- Main Generation Logic ---

print("1. Data Loading...")
df = pd.read_csv(CONFIG['data_path'])
token2idx = build_vocab(df['smiles'].tolist())
idx2token = {v: k for k, v in token2idx.items()}

# Cell Line List 확인
cell_lines = sorted(df['cell_line'].unique().tolist())
cell2idx = {name: i for i, name in enumerate(cell_lines)}

# PC3 (전립선암) 타겟 데이터 준비
print("Targeting Disease: Prostate Cancer (PC3)")
target_row = df[df['cell_line'] == 'PC3'].iloc[0]

# Gene Embedding
raw_gene = target_row['gene_e']
gene_values = parse_gene_string(raw_gene)
gene_embeds = []
for val in gene_values:
    gene_embeds.extend(tenfold_binary_embedding(val))

# Tensor 변환
expected_len = 978 * 10
if len(gene_embeds) != expected_len:
    gene_embeds = [0] * expected_len

gene_tensor = torch.tensor(gene_embeds, dtype=torch.float32).unsqueeze(0).to(DEVICE)
cell_idx = torch.tensor([cell2idx['PC3']], dtype=torch.long).to(DEVICE)

print("2. Model Loading...")
model = TransGEM(
    num_genes_dim=CONFIG['gene_dim'],
    num_cells=len(cell_lines),
    vocab_size=len(token2idx),
    d_model=CONFIG['d_model'], nhead=CONFIG['nhead'], num_layers=CONFIG['num_layers'], dim_feedforward=CONFIG['ff_dim']
).to(DEVICE)

if os.path.exists(CONFIG['model_path']):
    state_dict = torch.load(CONFIG['model_path'])
    model.load_state_dict(state_dict)
    print(f"✅ Checkpoint loaded: {CONFIG['model_path']}")
else:
    print(f"❌ Checkpoint NOT found at {CONFIG['model_path']}")
    raise FileNotFoundError("모델 파일 경로를 다시 확인해주세요!")

model.eval()

print("3. Generating Molecules...")
generated_smiles = []

# ✨ 여기서 진행바(Progress Bar)를 구경하시면 됩니다! ✨
for _ in tqdm(range(1000), desc="Creating New Drugs"):
    curr_seq = torch.tensor([[token2idx['<sos>']]], dtype=torch.long).to(DEVICE)
    
    with torch.no_grad():
        for _ in range(100): # Max Length
            output = model(cell_idx, gene_tensor, curr_seq)
            next_token_logits = output[:, -1, :]
            
            # Sampling
            probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, 1)
            
            curr_seq = torch.cat([curr_seq, next_token], dim=1)
            if next_token.item() == token2idx['<eos>']:
                break
    
    # Decoding
    indices = curr_seq[0].cpu().numpy()
    tokens = []
    for idx in indices:
        if idx == token2idx['<eos>']: break
        if idx not in [token2idx['<sos>'], token2idx['<pad>']]:
            tokens.append(idx2token[idx])
    
    selfies_str = "".join(tokens)
    try:
        smiles = sf.decoder(selfies_str)
        if smiles: generated_smiles.append(smiles)
    except:
        continue

# 저장
with open(CONFIG['output_file'], "w") as f:
    for s in generated_smiles:
        f.write(s + "\n")
        
print(f"🎉 Done! {len(generated_smiles)} molecules saved to {CONFIG['output_file']}")

1. Data Loading...
Building Vocabulary...


Tokenizing: 100%|██████████| 27378/27378 [00:09<00:00, 2765.88it/s]


Targeting Disease: Prostate Cancer (PC3)
2. Model Loading...
✅ Checkpoint loaded: checkpoints/model_epoch_200.pt
3. Generating Molecules...


Creating New Drugs: 100%|██████████| 1000/1000 [03:47<00:00,  4.40it/s]

🎉 Done! 1000 molecules saved to generated_molecules_PC3.txt
